* [x] Load Data
* [x] Create CNN 
* [x] Train CNN
* [ ] Visualize CNN
* [ ] Train Forecast using CNN
* [ ] Visualize Forecast CNN

In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import DataLoader

from fastai.basic_data import DataBunch
from fastai.basic_train import Learner
from fastai.callbacks import OneCycleScheduler
from fastai.basic_data import DatasetType

from util import convert_ts_data_to_cnn_ts_data,FastAICompatibleDataSet, dev_to_np
from autoencoder import Autoencoder

import datasets, converter


In [2]:
batch_size = 50
max_epoch = 1000

lr = 0.001
lr_max = 0.1

shuffle = True
drop_last = True

splits = [0.7,0.2,0.1]


num_jobs=4

device = "cpu"

In [3]:
data = datasets.HessianLoadDataset("../csv",use_cache=False,nr_parallel_file_reader=num_jobs,data_scaler=None)

# put data in variable for vizualization    
#orinial_data = data.original_data    #all data
training_data=data.train_data['targets'][0]
test_data=data.test_data['targets'][0]
validation_data=data.validation_data['targets'][0]


#read in residual data


residual_data =  pd.read_csv(f"residual_load.csv", sep=";", decimal=".",thousands=",", 
                             na_values=["-"], dtype={"Datum":str, "Uhrzeit":str}) 
residual_data.index = pd.to_datetime([f"{residual_data['Datum'][d]} {residual_data['Uhrzeit'][d]}" 
                                         for d in residual_data.index], format="%d.%m.%Y %H:%M:%S")
residual_data = residual_data.fillna(method="bfill")

residual_data = residual_data.drop(['Datum', 'Uhrzeit'],axis=1)

indices_train = pd.to_datetime(pd.date_range(start=data.train_data['targets'][0].index[0], end=data.train_data['targets'][0].index[-1], freq="H"))


train_residual = residual_data.loc[residual_data.index.intersection(indices_train)]['Residuallast']
train_residual=train_residual.resample('1H').asfreq()
train_residual=train_residual.interpolate()

indices_validation = pd.to_datetime(pd.date_range(start=data.validation_data['targets'][0].index[0], end=data.validation_data['targets'][0].index[-1], freq="H"))
validation_residual = residual_data.loc[residual_data.index.intersection(indices_validation)]['Residuallast']

indices_test = pd.to_datetime(pd.date_range(start=data.test_data['targets'][0].index[0], end=data.test_data['targets'][0].index[-1], freq="H"))
test_residual= residual_data.loc[residual_data.index.intersection(indices_test)]['Residuallast']


# create TimeFeatures    'Month of Year', 'Day of Month' , 
                        #'Day of Week' and 'Hour of Day'

TFC=converter.TimeFeatureCreator(year=False,
                                        month_of_year = True,
                                        week_of_year = False,
                                        day_of_year = False,
                                        day_of_month = True,
                                        day_of_week = True,
                                        hour_of_day = True,
                                        minute_of_hour = False,
                                        do_sin_cos_encoding =True)


#write time features into DataFrames, name columns and set index
Time_train=pd.DataFrame(TFC.transform(data.train_data["inputs"][0].index)) #Time Features for train data
Time_train.columns=TFC.extracted_features_name # naming columns
Time_train.index = pd.DatetimeIndex(data.train_data["inputs"][0].index.values, freq='1H') # set index

Time_validation=pd.DataFrame(TFC.transform(data.validation_data["inputs"][0].index)) #time features for validationd ata
Time_validation.columns=TFC.extracted_features_name
Time_validation.index = pd.DatetimeIndex(data.validation_data["inputs"][0].index.values, freq='1H')

Time_test=pd.DataFrame(TFC.transform(data.test_data["inputs"][0].index)) #time features for test data
Time_test.columns=TFC.extracted_features_name
Time_test.index = pd.DatetimeIndex(data.test_data["inputs"][0].index.values, freq='1H')

cos_sin_header = [x for x in Time_train.columns if ('sin' in x or 'cos' in x)]
Time_train = Time_train[cos_sin_header]
Time_validation = Time_validation[cos_sin_header]
Time_test = Time_test[cos_sin_header]

# Complement Input-Data with Time-Stamp-Data for training, validation and test
# note, that the Input-Data are the same for whole Dataset

input_train=pd.concat([data.train_data["inputs"][0],Time_train], axis=1)
input_validation=pd.concat([data.validation_data["inputs"][0],Time_validation], axis=1)
input_test=pd.concat([data.test_data["inputs"][0],Time_test], axis=1)

# define Reduced-Inputs for training, validation and test
# reduced Input-Data = Temperature, Hour of Day, Day of Week


reduced_features=['temp2m','day_of_week_sin', 'day_of_week_cos', 
                'hour_of_day_sin', 'hour_of_day_cos' ]

input_train_reduced=input_train[reduced_features].copy()
input_validation_reduced=input_validation[reduced_features].copy()
input_test_reduced=input_test[reduced_features].copy()

In [4]:

Time_train[cos_sin_header]

,month_of_year_sin,month_of_year_cos,day_of_month_sin,day_of_month_cos,day_of_week_sin,day_of_week_cos,hour_of_day_sin,hour_of_day_cos
2016-11-01 00:00:00,-0.5,0.866025,0.201299,0.979530,0.781831,0.623490,0.000000,1.000000e+00
2016-11-01 01:00:00,-0.5,0.866025,0.201299,0.979530,0.781831,0.623490,0.258819,9.659258e-01
2016-11-01 02:00:00,-0.5,0.866025,0.201299,0.979530,0.781831,0.623490,0.500000,8.660254e-01
2016-11-01 03:00:00,-0.5,0.866025,0.201299,0.979530,0.781831,0.623490,0.707107,7.071068e-01
2016-11-01 04:00:00,-0.5,0.866025,0.201299,0.979530,0.781831,0.623490,0.866025,5.000000e-01
...,...,...,...,...,...,...,...,...
2017-07-14 03:00:00,-0.5,-0.866025,0.299363,-0.954139,-0.433884,-0.900969,0.707107,7.071068e-01
2017-07-14 04:00:00,-0.5,-0.866025,0.299363,-0.954139,-0.433884,-0.900969,0.866025,5.000000e-01
2017-07-14 05:00:00,-0.5,-0.866025,0.299363,-0.954139,-0.433884,-0.900969,0.965926,2.588190e-01
2017-07-14 06:00:00,-0.5,-0.866025,0.299363,-0.954139,-0.433884,-0.900969,1.000000,6.123234e-17


In [5]:
# data = pd.read_csv('./data/wf1.csv')

# data.index = pd.to_datetime([x.replace('0000','2017').replace('0001','2018') for x in data['Time']])

# data = data.drop(['Time','ForecastingTime'],axis=1)

In [6]:
# def cnn_mse_loss(y_true, y_pred):
#     return np.mean(np.sqrt(np.mean(np.mean((y_pred-y_true)**2,axis=2),axis=0)))

In [7]:
# day_of_x_features = pd.DataFrame(np.array([data.index.month.values, data.index.dayofyear.values, data.index.dayofweek.values, data.index.weekofyear.values]).transpose(), columns=['MonthOfYear','DayOfYear','DayOfWeek','WeekOfYear'], index=data.index)

In [8]:
# all_data = pd.concat([data,day_of_x_features],axis=1)

In [9]:
# all_data = all_data/all_data.max()

In [10]:
# target = all_data['PowerGeneration']
# all_data = all_data.drop(['PowerGeneration'],axis=1)

In [11]:
# split_sizes = np.cumsum([int(x*all_data.shape[0]) for x in splits])

# train_data = convert_ts_data_to_cnn_ts_data(all_data.iloc[0:split_sizes[0]])
# val_data = convert_ts_data_to_cnn_ts_data(all_data.iloc[(split_sizes[1]+1):split_sizes[2]])
# test_data = convert_ts_data_to_cnn_ts_data(all_data.iloc[(split_sizes[0]+1):split_sizes[1]])


In [12]:
# val_data.shape

In [13]:
train_max = input_train.max()

In [14]:
train_max

temp2m                 301.200012
rhum2m                 100.000000
srfprs               99242.960938
cdclow                 100.000000
cdcmid                 100.000000
cdchig                 100.000000
cdctbl                 100.000000
cdctcl                 100.000000
totpre                  10.580000
presno                   1.000000
preice                   0.000000
swfldw                 732.000000
pblhgt                1967.040039
sundur               21600.000000
albedo                  58.700001
liftix                  22.700001
tozone                 470.299988
capest                2671.000000
cfrain                   1.000000
ww30mb                  24.325743
wd30mb                 359.843231
wwsig1                  18.566326
wdsig1                 359.600739
ww010m                  13.814692
wd010m                 359.593658
ww100m                  23.642803
wd100m                 359.795929
month_of_year_sin        1.000000
month_of_year_cos        1.000000
day_of_month_s

In [15]:
training_data

,Wirkleistung [kW]
DateTime,
2016-11-01 00:00:00,132.944165
2016-11-01 01:00:00,120.086362
2016-11-01 02:00:00,119.985972
2016-11-01 03:00:00,87.484272
2016-11-01 04:00:00,86.288306
...,...
2017-07-14 03:00:00,150.038222
2017-07-14 04:00:00,157.404051
2017-07-14 05:00:00,148.391339


In [16]:
input_train_scaled = input_train/train_max
input_validation_scaled = input_validation/train_max
input_test_scaled = input_train/train_max

In [17]:
for x in [input_test_scaled,input_validation_scaled,input_train_scaled]:
    x.drop(['preice'],inplace=True, axis=1)

(6128, 34)

In [37]:
def transform_to_sequence(data,timesteps=24):
    samples = data.shape[0]
    features = data.shape[1]
    return data[range(samples//timesteps * timesteps),:].reshape(-1,timesteps,features)

In [47]:
transform_to_sequence(training_data.values).shape

(255, 24, 1)

In [17]:
def check_nans(df):
    result = False
    if (type(df) == pd.DataFrame or type(df) == pd.Series):
        df = df.replace([np.inf, -np.inf], np.nan)
        result = df.isnull().any().any()
    elif type(df) == np.ndarray or type(df) == np.array:
        result = np.any(np.isnan(df)) or np.any(np.isinf(df))
    else: 
        result = True
    if (result):
        print("data is null/nan ")
        raise Exception("data is null/nan")
    else:
        return result


In [18]:
for i,input_data in enumerate([input_test, input_train, input_validation, input_train_reduced, input_validation_reduced, input_test_reduced,input_train_scaled,input_validation_scaled,input_test_scaled]):
    print(i)
    if check_nans(input_data):
        print(f"{i}th input data contains nan")
        raise Exception("can't continue with nan in input set")

0
1
2
3
4
5
6
7
8


In [19]:
train_dl = DataLoader(FastAICompatibleDataSet(input_train_scaled,input_train_scaled),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
val_dl = DataLoader(FastAICompatibleDataSet(input_validation_scaled,input_validation_scaled),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
test_dl = DataLoader(FastAICompatibleDataSet(input_test_scaled,input_test_scaled),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)


In [20]:
fc_train_dl = DataLoader(FastAICompatibleDataSet(input_train_scaled,training_data/training_data.max()),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
fc_val_dl = DataLoader(FastAICompatibleDataSet(input_validation_scaled,validation_data/training_data.max()),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
fc_test_dl = DataLoader(FastAICompatibleDataSet(input_test_scaled,test_data/training_data.max()),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)


In [48]:
def transform_to_sequence(data,timesteps=24):
    samples = data.shape[0]
    features = data.shape[1]
    return data[range(samples//timesteps * timesteps),:].reshape(-1,timesteps,features)

input_train_scaled_seq = transform_to_sequence(input_train_scaled.values)
input_validation_scaled_seq = transform_to_sequence(input_validation_scaled.values)
input_test_scaled_seq = transform_to_sequence(input_test_scaled.values)

target_train_seq = transform_to_sequence((training_data/training_data.max()).values)
target_val_seq = transform_to_sequence((validation_data/training_data.max()).values)
target_test_seq = transform_to_sequence((test_data/training_data.max()).values)

fc_train_dl_seq = DataLoader(FastAICompatibleDataSet(input_train_scaled_seq,target_train_seq),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
fc_val_dl_seq = DataLoader(FastAICompatibleDataSet(input_validation_scaled_seq,target_val_seq),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
fc_test_dl_seq = DataLoader(FastAICompatibleDataSet(input_test_scaled_seq,target_test_seq),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)


In [21]:
train_dl.dataset.to_np()[0].shape

(6128, 38)

In [22]:
db = DataBunch(train_dl, val_dl, test_dl=test_dl)
fc_db = DataBunch(fc_train_dl, fc_val_dl, test_dl=fc_test_dl)
fc_seq_db = DataBunch(fc_train_dl_seq, fc_val_dl_seq, test_dl=fc_test_dl_seq)

In [23]:
loss = torch.nn.MSELoss()

In [24]:
model = Autoencoder(input_train_scaled.shape[1],[30,20, 10]).to(device)

In [25]:
model

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=38, out_features=30, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=20, out_features=10, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=20, out_features=30, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=30, out_features=38, bias=True)
  )
)

In [26]:
def init_model(m,init_function=torch.nn.init.xavier_normal_):
    if type(m) == torch.nn.Linear:
        init_function(m.weight)
        m.bias.data.fill_(0.1)
    if isinstance(m, torch.nn.Conv1d) or isinstance(m, torch.nn.ConvTranspose1d):
        init_function(m.weight.data)
        m.bias.data.fill_(0.1)

In [27]:
model.apply(lambda m: init_model(m))

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=38, out_features=30, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=20, out_features=10, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=20, out_features=30, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=30, out_features=38, bias=True)
  )
)

In [28]:
# db.train_ds.

In [ ]:
learner = Learner(db,model,loss_func=loss)
cb = OneCycleScheduler(learner, lr_max=lr_max)

learner.fit(max_epoch,lr = lr)

epoch,train_loss,valid_loss,time
0,0.133598,0.141878,00:00
1,0.092900,0.112172,00:00
2,0.065562,0.096716,00:00
3,0.052986,0.086476,00:00
4,0.045285,0.083749,00:00
5,0.039102,0.086109,00:00
6,0.034995,0.093324,00:00
7,0.032981,0.090167,00:00
8,0.024361,0.079001,00:00
9,0.019251,0.086160,00:00


In [42]:
y_pred,y_true = learner.get_preds(DatasetType.Valid)
y_pred = dev_to_np(y_pred)
y_true = dev_to_np(y_true)

In [43]:
np.sqrt(np.mean(np.power(y_true-y_pred,2)))

0.28570512

In [ ]:
import copy

In [ ]:
class AutoLSTM(torch.nn.Module):
    
    def __init__(self, autoencoder, nr_lstm_stages,output_size=1, levels_to_retrain=1, layer_type=torch.nn.Linear):
        super(AutoLSTM, self).__init__()
        
        self.encoder = copy.deepcopy(autoencoder.encoder)
        self.nr_lstms = nr_lstm_stages
        self.encoder_fix_n_level = levels_to_retrain
        self.encoder_layer_type = layer_type
        
        # fix layers
        layers_fixed = 0
        encoder_output_size = 0
        for i,param in enumerate(self.encoder.parameters()):
            
            
            if layers_fixed < (len(self.encoder) - levels_to_retrain-1):
                param.requires_grad = False
            else:
                param.requires_grad = True
            encoder_output_size = param.shape[0]
            layers_fixed = layers_fixed + 1
            
        self.autolstm = torch.nn.Sequential(*[self.encoder, torch.nn.Linear(encoder_output_size, 1)])
        
    def forward(self,input):
        
        x = self.autolstm(input)
        
        return x


In [ ]:
autolstm = AutoLSTM(model,2)

In [ ]:
from fastai.metrics import root_mean_squared_error

In [ ]:
autolstm_learner = Learner(fc_db,autolstm,loss_func=loss,metrics=root_mean_squared_error)
autolstm_cb = OneCycleScheduler(learner, lr_max=lr_max)

autolstm_learner.fit(max_epoch,callbacks=[cb],lr = lr)

In [ ]:
autolstm_learner.recorder.plot_metrics()

In [ ]:
y_pred,y_true = autolstm_learner.get_preds(DatasetType.Valid)
y_pred = dev_to_np(y_pred)
y_true = dev_to_np(y_true)

In [ ]:
np.sqrt(np.mean(np.power(y_true-y_pred,2)))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(y_true, label = "true")
plt.plot(y_pred,alpha=0.5, label="predicted")

plt.legend()

In [62]:
a = torch.arange(0,1000).view(10,10,10)

In [67]:
b = a.flatten().view(100,10)

In [68]:
b.view(10,10,10)

tensor([[[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19],
         [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39],
         [ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
         [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79],
         [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99]],

        [[100, 101, 102, 103, 104, 105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
         [120, 121, 122, 123, 124, 125, 126, 127, 128, 129],
         [130, 131, 132, 133, 134, 135, 136, 137, 138, 139],
         [140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
         [150, 151, 152, 153, 154, 155, 156, 157, 158, 159],
         [160, 161, 16